# 元编程（Meta-Programming）

什么是元编程呢？

简而言之，代码是可以操作的对象，我可以通过元编程来**自动生成代码， 并执行代码**

## 创建表达式

### 通过字符串创建表达式

In [1]:
ex1 = Meta.parse("1 + 1")

:(1 + 1)

In [2]:
eval(ex1)

2

In [3]:
ex2 = Meta.parse("1 + aaaa")
ex2 # 虽然我们不知道a是几，如果scope中aaaa有定义，那么ex2是可以执行的

:(1 + aaaa)

In [4]:
eval(ex2)

LoadError: UndefVarError: `aaaa` not defined

In [5]:
aaaa = 100
eval(ex2)

101

In [6]:
dump(ex1)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1


### 通过`Expr`手动创建

In [7]:
Expr(:call, :+, 1, 1)

:(1 + 1)

### 通过`quote`创建多个表达式

In [8]:
ex3 = quote
    x = 1
    y = 2
end

quote
    #= In[8]:2 =#
    x = 1
    #= In[8]:3 =#
    y = 2
end

In [9]:
dump(ex3)

Expr
  head: Symbol block
  args: Array{Any}((4,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[8]
    2: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol x
        2: Int64 1
    3: LineNumberNode
      line: Int64 3
      file: Symbol In[8]
    4: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol y
        2: Int64 2


可以看出`quote`通过`array`嵌套了多个表达式， 他的`head`是`:block`（中间有两行是注释）

In [10]:
esc(ex3)

:($(Expr(:escape, quote
    #= In[8]:2 =#
    x = 1
    #= In[8]:3 =#
    y = 2
end)))

### 直接通过`:()`创建

In [11]:
:(1+1) == Expr(:call, :+, 1, 1)

true

In [12]:
ex2 = :(2 + 3 * a + b) 

:(2 + 3a + b)

In [13]:
dump(ex2)

Expr
  head: Symbol call
  args: Array{Any}((4,))
    1: Symbol +
    2: Int64 2
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Int64 3
        3: Symbol a
    4: Symbol b


julia的表达式通常尽可以包含`符号、子表达式、字面值`， 上式中`+, b`是符号， `:(3 * a)`是子表达式， `2`是字面值，上式是一个嵌套的表达式， **`a, b`的值还不知道， 在这里被当做符号对待**

<font style="color:purple;font-size:14pt;">如果`a, b`已知， 那么通过`$`进行插值， 可以使构造的表达式中的`a, b`替换为变量`a, b`的实际值</font>

In [14]:
a = 1
b = 1
:(2 + 3 * a + b) # no $

:(2 + 3a + b)

In [15]:
# 仍然是可以求值的
eval(:(2 + 3 * a + b))

6

In [16]:
:(2 + 3 * $a + $b) #！！ with $ 

:(2 + 3 * 1 + 1)

In [17]:
# 很自然的可以求值
eval(:(2 + 3 * $a + $b))

6

In [18]:
# splatting interploration

args = [:x, :y, :z]
:(f(1, $(args...)))

:(f(1, x, y, z))

可以看出来整个工作的流程就是， 利用`Expr`表达式生成代码， 然后使用`eval`函数求值， 达到自动生成代码， 运行代码的目的

## 宏

宏像是一个函数， 但是又与函数不太相同， **宏会返回一个Expr表达式， 这个表达式不经过`eval`直接自动运行**, `@macroexpand`可以看到生成的表达式。

宏相当于：
1. 构造一个函数`function`，该函数的返回为表达式
2. 通过`eval()`执行该表达式

宏的作用：按照一定规则重构代码，如`@threads`...
+ Domain Specific Language, e.g., JuMP.jl, Turing.jl

### 定义宏

In [19]:
macro sayhello()
    return :(println("Hello World!"))
end

@sayhello (macro with 1 method)

In [20]:
@sayhello

Hello World!


In [21]:
macro sayhello0(name)
    return :(println("Hello World! ", name))
end
macro sayhello1(name)
    return :(println("Hello World! ", $name))
end

@sayhello1 (macro with 1 method)

In [22]:
@macroexpand @sayhello0 "XJZ"

:(Main.println("Hello World! ", Main.name))

In [23]:
@macroexpand @sayhello1 "XJZ"

:(Main.println("Hello World! ", "XJZ"))

*Notes*
+ `sayhello1(name)`这个宏中用到了插值`$name`, 表明宏将`name`这个位置参数的值插入到构造的表达式中
+ `sayhello0(name)`这个宏中直接使用`name`, 则宏将`name`理解为symbol加入到的表达式中，实际执行时为`Main.name`，即全局变量

因此，在构造`Expr`的过程中，使用任何函数参数都要进行插值🐮

### 理解宏的运行

In [24]:
macro twostep(arg...)
    println("I execute at parse time. The argument is: ", arg...)
    return :(println("I execute at runtime. The argument is: ", $(arg...)))
end

@twostep (macro with 1 method)

In [25]:
@macroexpand @twostep(1, 2, 3)

I execute at parse time. The argument is: 123


:(Main.println("I execute at runtime. The argument is: ", 1, 2, 3))

可以看出， 仅生成表达式时， 会执行中间定义的命令，即**生成表达式也属于执行命令**

执行宏命令时， 会运行宏内部的的所有语句命令， 以及"`eval`"该宏返回的表达式

In [26]:
@twostep 1, 2, 3

I execute at parse time. The argument is: (1, 2, 3)
I execute at runtime. The argument is: (1, 2, 3)


### 卫生宏

卫生宏，英文为`Hygienic Macro`， 指宏内定义的变量与宏运行环境中的变量不冲突， Julia默认帮你做到了这一点(默认定义的变量为`local`)， 但是有些情况我们希望宏能够修改运行环境中的一些变量， 那么就可以使用`global`，声明该变量为全局变量， 这样可以修改全局作用域中的变量。

然而存在一种情况， 例如函数内使用了宏， 这个宏想修改函数内的变量的值而不是全局作用域的值， 这样怎么搞？， 答`esc()`

In [27]:
# 局部变量， 运行环境内的变量没有任何影响
# 默认quote内定义的为局部变量
x = 0.1
y = 0.2
macro envtest1()
    return quote
         x = 1 # 前边隐藏了local
         y = 2 # 前边隐藏了local
    end
end
(@envtest1, x, y)

(2, 0.1000, 0.2000)

In [28]:
# 可以看出Julia对局部变量进行了重命名保证唯一性， 这样局quote中定义的变量就不会与全局变量冲突了
@macroexpand @envtest1

quote
    #= In[27]:7 =#
    var"#54#x" = 1
    #= In[27]:8 =#
    var"#55#y" = 2
end

In [29]:
# 另一种写法， 修改全局变量x的值， y为局部变量
x = 0.1
y = 0.2
macro envtest2()
    expr = quote
         global x = 1
         y = 2
    end
    return expr
end
(@envtest2, x, y)

(2, 1, 0.2000)

In [30]:
# 仅仅y被重命名了
@macroexpand @envtest2

quote
    #= In[29]:6 =#
    global x = 1
    #= In[29]:7 =#
    var"#57#y" = 2
end

In [31]:
# 以纳秒（ns）的形式的获取当前时间， 内置函数
time_ns()
macro envtest3(ex)
    return quote
        t1 = time_ns()
        $(ex)
        t2 = time_ns()
        elapsed = (t2 - t1)/1e9
        println("elapsed time is ", elapsed)
        println(t1, "\n", t2)
    end
end

@envtest3 (macro with 1 method)

In [32]:
@macroexpand @envtest3 z = 1 + 2 # 这里的z竟然也是局部变量， 相当于把z加入了quote中， 也是在quote中定义为局部变量

quote
    #= In[31]:5 =#
    var"#58#t1" = Main.time_ns()
    #= In[31]:6 =#
    var"#59#z" = 1 + 2
    #= In[31]:7 =#
    var"#60#t2" = Main.time_ns()
    #= In[31]:8 =#
    var"#61#elapsed" = (var"#60#t2" - var"#58#t1") / 1000000000.0000
    #= In[31]:9 =#
    Main.println("elapsed time is ", var"#61#elapsed")
    #= In[31]:10 =#
    Main.println(var"#58#t1", "\n", var"#60#t2")
end

In [33]:
function test_env()
    t1 = 1
    t2 = 2
    return t2 + t2
end

test_env (generic function with 1 method)

In [34]:
@macroexpand @envtest3 test_env()

quote
    #= In[31]:5 =#
    var"#62#t1" = Main.time_ns()
    #= In[31]:6 =#
    Main.test_env()
    #= In[31]:7 =#
    var"#63#t2" = Main.time_ns()
    #= In[31]:8 =#
    var"#64#elapsed" = (var"#63#t2" - var"#62#t1") / 1000000000.0000
    #= In[31]:9 =#
    Main.println("elapsed time is ", var"#64#elapsed")
    #= In[31]:10 =#
    Main.println(var"#62#t1", "\n", var"#63#t2")
end

看以上宏展开结果， quote中所有宏调用的函数都是`Main.function`，即全局作用域中的函数

In [35]:
macro envtest3(ex)
    return quote
        t1 = time_ns()
        $(esc(ex))
        t2 = time_ns()
        elapsed = (t2 - t1)/1e9
        println("elapsed time is ", elapsed)
        println(t1, "\n", t2)
    end
end

@envtest3 (macro with 1 method)

In [36]:
@macroexpand @envtest3 test_env()

quote
    #= In[35]:3 =#
    var"#65#t1" = Main.time_ns()
    #= In[35]:4 =#
    test_env()
    #= In[35]:5 =#
    var"#66#t2" = Main.time_ns()
    #= In[35]:6 =#
    var"#67#elapsed" = (var"#66#t2" - var"#65#t1") / 1000000000.0000
    #= In[35]:7 =#
    Main.println("elapsed time is ", var"#67#elapsed")
    #= In[35]:8 =#
    Main.println(var"#65#t1", "\n", var"#66#t2")
end

加入`esc`以后， `Main.`被去掉了， 即**当前作用域能看到的`test_env()`我都可以去调用**

In [37]:
# 案例
macro zero_scope()
    return esc(:(xx = 0))
end
macro zero_global()
    return quote
        global xx = 0
    end
end

@zero_global (macro with 1 method)

In [38]:
@macroexpand @zero_scope

:(xx = 0)

In [39]:
xx = 100
function change_scope()
    xx = 99
    @zero_scope
    return xx
end
change_scope(), xx # xx在全局作用域中还是100，不过在change_scope()的这个作用域中被改为了0

(0, 100)

In [40]:
xx = 100
function change_global()
    @zero_global
    return xx
end
change_global(), xx # xx在全局作用域中变成了0，相当于执行了Main。xx = 0

(0, 0)

这可案例可以清楚的看到， `global`声明会直接修改全局作用域的变量， 这样其实不太好， 而`esc()`仅会修改上层作用域的变量， 毕竟没有`Main.`, 宏出现在哪个作用域里， 他就会调用哪个作用域里都可以看到的函数

### 案例一：assert

In [41]:
macro assert(ex)
    return :($ex ? nothing : AssertionError("not equal"))
end

@assert (macro with 1 method)

In [42]:
@assert 1 == 12

AssertionError("not equal")

In [43]:
@macroexpand @assert 1 == 2.0

:(if 1 == 2.0000
      Main.nothing
  else
      Main.AssertionError("not equal")
  end)